<div class="alert alert-block alert-warning">
	<b>Warning:</b> Before running below cells please make sure you have API key. 
                Please see 
	<a href="https://github.com/heremaps/here-location-services-python/blob/master/docs/notebooks/README.md">README.md</a> for more info on API key.

</div>

# HERE Matrix Routing API 

[Matrix Routing API v8](https://developer.here.com/documentation/matrix-routing-api/8.3.0/dev_guide/index.html)

HERE Matrix Routing service calculates the travel duration and/or distances among multiple origins and destinations.
Matrix routing can be done with synchronous and asynchrounous requests.

Calculation of routing matrices in one of the following modes:

- Flexible
- Region
- Profile

The values of the `region_definition` and `profile` parameters determine which mode is used. The following table describes the capabilities and limitations of each mode.

|Mode | region_definition parameter |profile parameter provided?|Custom Options & Time Awareness (incl. live traffic)|Unlimited region|
|:-------|:------------------------------------|:---------|:-----------|:--------------------------------------------------------------------------|
|Flexible|world                                |no        |yes         |yes                                                                        |
|Region  |one of:<br />Circle<br />boundingBox<br />polygon<br />autoCircle|no        |yes         |no<br />origins and destinations must be within a region of max 400 km diameter|              
|Profile |world                                |yes       |no          |yes           |



Note that the combination of specifying a profile along with a region_definition not equal to world is not allowed.

## Flexible Mode

Flexible mode provides capabilities such as Custom options, Time Awareness (including Live Traffic), Unlimited Region
but it has Limited Matrix Size.
Given a list of origins and a list of destinations, the service computes the shortest travel times or distances between every pair of origin and destination.
These results make up the entries of the [routing matrix](https://developer.here.com/documentation/matrix-routing-api/8.3.0/dev_guide/topics/concepts/matrix.html).

In order to provide support for custom routing options, time awareness, and routes of arbitrary length, Flexible Mode cannot benefit from the optimizations that give Region and Profile modes their high performance. Due to this performance limitation, Flexible Mode requests are limited to:


at most 15 origins and 100 destinations (15 x 100)
or at most 100 origins and 1 destination (100 x 1)

### Formulating a request
Flexible Mode is utilized when:

the region definition  = WorldRegion
and no profile parameter is specified
The service applies live and historical traffic information unless explicitly disabled by setting departureTime to the special value any.

Below is an example of a 3x3 matrix request with the following origins and destinations:

- San Francisco at (37.76, -122.42)
- New York at (40.63, -74.09)
- Austin at (30.26, -97.74)

In [ ]:
import os

from here_location_services import LS
from here_location_services.config.matrix_routing_config import (
    WorldRegion,
    MATRIX_ATTRIBUTES,
)
from here_map_widget import Map, Marker, GeoJSON

In [ ]:
LS_API_KEY = os.environ.get("LS_API_KEY")  # Get API KEY from environment.

In [ ]:
m = Map(api_key=LS_API_KEY, center=[39.0794, -99.0943], zoom=4.2)
sf_marker = Marker(lat=37.76, lng=-122.42)
ny_marker = Marker(lat=40.63, lng=-74.09)
au_marker = Marker(lat=30.26, lng=-97.74)
m.add_object(sf_marker)
m.add_object(ny_marker)
m.add_object(au_marker)
m

In [ ]:
# Create Location Services object using API KEY.

ls = LS(api_key=LS_API_KEY)

In [ ]:
origins = [
    {"lat": 37.76, "lng": -122.42},
    {"lat": 40.63, "lng": -74.09},
    {"lat": 30.26, "lng": -97.74},
]
region_definition = WorldRegion()
matrix_attributes = [MATRIX_ATTRIBUTES.distances, MATRIX_ATTRIBUTES.travelTimes]

In [ ]:
result = ls.matrix(
    origins=origins,
    region_definition=region_definition,
    matrix_attributes=matrix_attributes,
)

In [ ]:
result.matrix

In [ ]:
distance_matrix= result.to_distnaces_matrix()
distance_matrix

In [ ]:
travel_times_matrix= result.to_travel_times_matrix()
travel_times_matrix

## Region Mode

This section refers to calculating matrices with custom options using a limited sized region.
Region is limited to max. 400km diameter.
By restricting the calculation to a specific region of at most 400 km diameter, it is possible to specify different options to take into account during calculation. The service applies live and historical traffic information unless explicitly disabled by setting `departure_time` to the special value `any`.

Region Mode supports:
- Custom options
- Time Awareness (including Live Traffic), using a snapshot of time at departure
- Matrix Sizes up to 10,000 x 10,000
- Region limited to max. 400km diameter

### BoundingBox region definition
Below is an example of a simple 3 x 3 matrix in Berlin, Germany with these origins and destinations:

- Alexanderplatz at (52.52103, 13.41268)
- Brandenburg Gate at (52.51628, 13.37771)
- Tempelhof Field at (52.47342, 13.40357)

To calculate a car distance matrix, you can use the below code. 
Since the request does not specify a destinations list, the origins are taken as destinations and the resulting matrix is a 3 x 3 matrix. 
The region definition is a bounding box around the points with a small margin added to be able to properly route in the vicinity of the points. 
By default, the service calculates a travel times matrix, but since we want to get distances in the response instead of times, the request specifies the `matrix_attributes` property with the value `distances`.

In [ ]:
from here_map_widget import Map, Bbox, Rectangle, Marker
import os

m = Map(api_key=os.environ['LS_API_KEY'], center=[52.5034, 13.4079], zoom=11.4)

style = {"strokeColor": "#829", "lineWidth": 4}

bbox = Bbox(top=52.53, left=13.35, bottom=52.46, right=13.42)
rectangle = Rectangle(bbox=bbox, style=style, draggable=True)
m.add_object(rectangle)
alex_marker = Marker(lat=52.52103, lng=13.41268)
bran_marker = Marker(lat=52.51628, lng=13.37771)
temp_marker = Marker(lat=52.47342, lng=13.40357)
m.add_object(alex_marker)
m.add_object(bran_marker)
m.add_object(temp_marker)
m

In [ ]:
from here_location_services import LS
from here_location_services.config.matrix_routing_config import (
    BoundingBoxRegion,
    AutoCircleRegion,
    MATRIX_ATTRIBUTES,
    PROFILE,
    WorldRegion
)

LS_API_KEY = os.environ.get("LS_API_KEY")  # Get API KEY from environment.
ls = LS(api_key=LS_API_KEY)  # Create Location Services object using API KEY.

In [ ]:
origins = [
    {"lat": 52.52103, "lng": 13.41268},
    {"lat": 52.51628, "lng": 13.37771},
    {"lat": 52.47342, "lng": 13.40357},
]
region_definition = BoundingBoxRegion(north=52.53, south=52.46, west=13.35, east=13.42)
matrix_attributes = [MATRIX_ATTRIBUTES.distances]

In [ ]:
result = ls.matrix(
    origins=origins,
    region_definition=region_definition,
    matrix_attributes=matrix_attributes,
    async_req=True
)

In [ ]:
result.matrix

In [ ]:
distance_matrix= result.to_distnaces_matrix()
distance_matrix

### AutoCircle region definition

Instead of defining a bounding box around the origins, you can request for a circle to be automatically derived. The request below is for the same as the one above, but using the AutoCircle feature. Since the margin field is not provided, the service uses a default value of 10 kilometers.

In [ ]:
region_definition = AutoCircleRegion()

result = ls.matrix(
    origins=origins,
    region_definition=region_definition,
    matrix_attributes=matrix_attributes,
    async_req=True
)

In [ ]:
resp = result.response
resp

In [ ]:
auto_circle = resp["regionDefinition"]
center = auto_circle["center"]
radius = auto_circle["radius"]

In [ ]:
# AutoCircle Region visualisation

from here_map_widget import Circle, Point

m = Map(api_key=os.environ["LS_API_KEY"], center=[center["lat"],center["lng"]] , zoom=10.4)

style = {"strokeColor": "#829", "lineWidth": 4}

point = Point(lat=center["lat"], lng=center["lng"])
circle = Circle(center=point, radius=radius, style=style)
m.add_object(circle)
alex_marker = Marker(lat=52.52103, lng=13.41268)
bran_marker = Marker(lat=52.51628, lng=13.37771)
temp_marker = Marker(lat=52.47342, lng=13.40357)
m.add_object(alex_marker)
m.add_object(bran_marker)
m.add_object(temp_marker)
m

## Profile Mode

Profile mode supports:
- unlimited region
-  Matrix Sizes up to 10,000 x 10,000

Profile mode doesnot supports: 
- Custom Options
- Time Awareness (including Live Traffic)

This section refers to calculating matrices with routes of arbitrary length, using one of the supported profiles. If you want to define custom options, see `Flexible Mode`.

The special variant `world` needs to be set as region definition. No additional request options or departure_time can provided except for matrix_attributes.
Below is an example of a 7 x 7 matrix request with these origins and destinations:

- Berlin at (52.54, 13.40)
- Kyiv at (50.43, 30.52)
- London at (51.50, -0.08)
- Madrid at (40.40, -3.68)
- Moscow at (55.75, 37.60)
- Paris at (48.87, 2.33)
- Rome at (41.90, 12.48)

To calculate a car distance matrix, you can use the code below. Since the request does not specify the destinations array, the origins are taken as destinations and the resulting matrix is a 7 x 7 matrix. The region definition is the special variant world. In the request, we use the profile carFast which uses transport mode car and optimizes the route calculations for travel time. By default the service calculates a travel times matrix, but since we want to get distances in the response instead of times, the request specifies the matrix_attributes property with the value distances.

In [ ]:
from here_map_widget import Map, Bbox, Rectangle, Marker
import os

m = Map(api_key=os.environ["LS_API_KEY"], center=[49.6193, 13.5609], zoom=4.2)

berlin_marker = Marker(lat=52.54, lng=13.40)
kyiv_marker = Marker(lat=50.43, lng=30.52)
london_marker = Marker(lat=51.50, lng=-0.08)
madrid_marker = Marker(lat=40.40, lng=-3.68)
moscow_marker = Marker(lat=55.75, lng=37.60)
paris_marker = Marker(lat=48.87, lng=2.33)
rome_marker = Marker(lat=41.90, lng=12.48)
m.add_objects(
    [
        berlin_marker,
        kyiv_marker,
        london_marker,
        madrid_marker,
        moscow_marker,
        paris_marker,
        rome_marker,
    ]
)
m

In [ ]:
origins = [
    {"lat": 52.54, "lng": 13.40},
    {"lat": 50.43, "lng": 30.52},
    {"lat": 51.50, "lng": -0.08},
    {"lat": 40.40, "lng": -3.68},
    {"lat": 55.75, "lng": 37.60},
    {"lat": 48.87, "lng": 2.33},
    {"lat": 41.90, "lng": 12.48},
]

profile = PROFILE.carFast
region_definition = WorldRegion()
matrix_attributes = [MATRIX_ATTRIBUTES.distances]

In [ ]:
result = ls.matrix(
    origins=origins,
    region_definition=region_definition,
    matrix_attributes=matrix_attributes,
    async_req=True
)

The response corresponds to this matrix with entries in meters:

In [ ]:
result.to_distnaces_matrix()